# How to match? 

## 0. Import libraries 

In [1]:
import pandas as pd
import warnings
import numpy as np
from lightautoml.addons.hypex import Matcher

warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


/Users/20224761/PycharmProjects/matcher/lightautoml/ml_algo/dl_model.py:41: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/Users/20224761/PycharmProjects/matcher/lightautoml/text/nn_model.py:22: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/Users/20224761/PycharmProjects/matcher/lightautoml/text/dl_transformers.py:25: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")


## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [2]:
# Simulating dataset with known effect size
num_users = 10000
num_months = 12

signup_months = np.random.choice(np.arange(1, num_months), num_users) * np.random.randint(0,2, size=num_users) # signup_months == 0 means customer did not sign up
df = pd.DataFrame({
    'user_id': np.repeat(np.arange(num_users), num_months),
    'signup_month': np.repeat(signup_months, num_months), # signup month == 0 means customer did not sign up
    'month': np.tile(np.arange(1, num_months+1), num_users), # months are from 1 to 12
    'spend': np.random.poisson(500, num_users*num_months) #np.random.beta(a=2, b=5, size=num_users * num_months)*1000 # centered at 500
})
# A customer is in the treatment group if and only if they signed up
df["treat"] = df["signup_month"]>0
# Simulating an effect of month (monotonically decreasing--customers buy less later in the year)
df["spend"] = df["spend"] - df["month"]*10
# Simulating a simple treatment effect of 100
after_signup = (df["signup_month"] < df["month"]) & (df["treat"])
df.loc[after_signup,"spend"] = df[after_signup]["spend"] + 100
df.head()

,user_id,signup_month,month,spend,treat
0,0,0,1,480,False
1,0,0,2,459,False
2,0,0,3,465,False
3,0,0,4,455,False
4,0,0,5,423,False


In [3]:
# Setting the signup month (for ease of analysis)
i = 3
df_i_signupmonth = (
    df[df.signup_month.isin([0, i])]
    .groupby(["user_id", "signup_month", "treat"])
    .apply(
        lambda x: pd.Series(
            {
                "pre_spends": x.loc[x.month < i, "spend"].mean(),
                "post_spends": x.loc[x.month > i, "spend"].mean(),
            }
        )
    )
    .reset_index()
)
df_i_signupmonth

,user_id,signup_month,treat,pre_spends,post_spends
0,0,0,False,469.5,416.333333
1,1,0,False,471.0,414.333333
2,5,0,False,482.0,409.222222
3,9,0,False,461.0,418.777778
4,10,0,False,503.0,418.777778
...,...,...,...,...,...
5424,9991,0,False,469.0,424.777778
5425,9995,0,False,454.0,410.555556
5426,9996,0,False,475.0,420.333333
5427,9997,0,False,530.5,418.888889


In [4]:
# Additional category features
gender = np.random.choice(a=[0,1], size=df_i_signupmonth.user_id.nunique())
age = np.random.choice(a=range(18, 70), size=df_i_signupmonth.user_id.nunique())
industry = np.random.choice(a=range(1, 3), size=df_i_signupmonth.user_id.nunique())
df_i_signupmonth['age'] = age
df_i_signupmonth['is_male'] =  gender
df_i_signupmonth['industry'] =  industry
df_i_signupmonth['industry'] = df_i_signupmonth['industry'].astype('str')
df_i_signupmonth['treat'] = df_i_signupmonth['treat'].astype(int)
df_i_signupmonth.head()

,user_id,signup_month,treat,pre_spends,post_spends,age,is_male,industry
0,0,0,0,469.5,416.333333,58,0,2
1,1,0,0,471.0,414.333333,33,0,1
2,5,0,0,482.0,409.222222,45,1,2
3,9,0,0,461.0,418.777778,30,1,2
4,10,0,0,503.0,418.777778,40,1,1


In [5]:
df_i_signupmonth.columns

Index(['user_id', 'signup_month', 'treat', 'pre_spends', 'post_spends', 'age',
       'is_male', 'industry'],
      dtype='object')

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [6]:
info_col = ['user_id', 'signup_month']

outcome = 'post_spends'
treatment = 'treat'

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [7]:
# Standard model with base parameters
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment, info_col=info_col)
results, quality_results = model.estimate()

  0%|          | 0/5429 [00:00<?, ?it/s]

### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [8]:
# group_col is used for strict comparison of categorical features.  
# In our case there is only one attribute  
# If there are several such attributes, you should make one of them and use it

group_col = "industry"

In [9]:
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment,
                info_col=info_col, group_col=group_col)
results, quality_results = model.estimate()

  0%|          | 0/4 [00:00<?, ?it/s]

### 2.3 Matching but you don't know which features are better to use  
This is the easiest way to initialize and calculate metrics on a Matching task  
Use this method when you want to select the most important features and make matching based on them. 

In [10]:
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment,
                info_col=info_col, group_col=group_col)

In [11]:
features_importance = model.lama_feature_select()
features_importance

,Feature,Importance
0,pre_spends,3.792544e+06
3,industry,3.792544e+06
1,age,2.538178e+06
2,is_male,2.782002e+05


In [12]:
features = features_importance['Feature'].to_list()

In [13]:
# You can use both variant that you like. So you can just list the features to be matched

#results, quality_results = model.estimate(features=features_importance[:3])
results, quality_results = model.estimate(features=features[:3])

  0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
model.matcher.df_matched

,index,pre_spends,age,is_male,industry,pre_spends_matched,age_matched,is_male_matched,index_matched,industry_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,4977,478.0,43,0,1,477.0,43.0,0.5,"[363, 1736]",1,531.333333,421.777778,109.544838,1,0
1,4978,459.0,48,0,1,457.5,49.0,0.0,[52],1,522.888889,424.555556,98.326824,1,0
2,4979,527.0,55,0,1,525.0,56.0,1.0,[900],1,503.888889,423.000000,80.877021,1,0
3,4980,496.0,62,0,1,496.5,62.0,0.0,[608],1,515.444444,421.444444,94.005359,1,0
4,4981,498.0,30,0,1,498.0,30.0,0.0,[475],1,519.888889,431.000000,88.888889,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4972,4972,478.0,63,1,2,478.5,61.0,1.0,[283],2,417.333333,519.666667,102.352429,0,1
4973,4973,469.0,48,1,2,469.5,47.0,0.0,[304],2,424.777778,525.777778,101.016516,0,1
4974,4974,454.0,37,0,2,452.5,35.0,0.0,[333],2,410.555556,517.777778,107.185576,0,1
4975,4975,475.0,64,1,2,474.0,63.0,0.0,[394],2,420.333333,529.111111,108.752487,0,1


## 3. Results  
### 3.1 ATE, ATT, ATC

In [15]:
# model.matcher.results
results

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,100.253309,0.535137,0.0,99.204440,101.302178
ATC,100.242588,0.545294,0.0,99.173811,101.311364
ATT,100.371367,0.655610,0.0,99.086371,101.656362


### 3.2 SMD, PSI, KS-test, repeats

In [16]:
# matching quality result - SMD
model.quality_result

{'psi':         column_treated  anomaly_score_treated check_result_treated  \
 0          age_treated                   0.00                   OK   
 1     industry_treated                   0.00                   OK   
 2  post_spends_treated                  16.11                  NOK   
 3   pre_spends_treated                   0.00                   OK   
 
         column_untreated  anomaly_score_untreated check_result_untreated  
 0          age_untreated                     0.01                     OK  
 1     industry_untreated                     0.00                     OK  
 2  post_spends_untreated                     8.24                    NOK  
 3   pre_spends_untreated                     0.01                     OK  ,
 'ks_test':             match_control_to_treat  match_treat_to_control
 age                       1.000000                0.509011
 is_male                   0.057995                0.001526
 pre_spends                1.000000                0.312744,
 's

In [17]:
# matching quality result - PSI
model.quality_result['psi']

,column_treated,anomaly_score_treated,check_result_treated,column_untreated,anomaly_score_untreated,check_result_untreated
0,age_treated,0.00,OK,age_untreated,0.01,OK
1,industry_treated,0.00,OK,industry_untreated,0.00,OK
2,post_spends_treated,16.11,NOK,post_spends_untreated,8.24,NOK
3,pre_spends_treated,0.00,OK,pre_spends_untreated,0.01,OK


In [18]:
# matching quality result - KS-test

model.quality_result['ks_test']

,match_control_to_treat,match_treat_to_control
age,1.000000,0.509011
is_male,0.057995,0.001526
pre_spends,1.000000,0.312744


In [19]:
# matching quality result - repeats
model.quality_result['repeats']

{'match_control_to_treat': 0.93, 'match_treat_to_control': 0.09}

### 3.3 Validation
validate result with placebo treatment  
our new effect size is close to zero (it should be)

In [20]:
 
model.validate_result()
 

  0%|          | 0/10 [00:00<?, ?it/s]

{'post_spends': [100.15633648224275, 0.2918322510903185]}

In [21]:
model.matcher.df_matched

,index,pre_spends,age,is_male,industry,random_feature,pre_spends_matched,age_matched,is_male_matched,random_feature_matched,index_matched,industry_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,4977,478.0,43,0,1,-1.023656,479.0,42.0,0.0,-1.057134,[160],1,531.333333,428.222222,103.125354,1,0
1,4978,459.0,48,0,1,1.247216,457.0,45.0,0.0,1.107004,[2176],1,522.888889,426.000000,96.838339,1,0
2,4979,527.0,55,0,1,1.016046,525.0,56.0,1.0,1.095232,[900],1,503.888889,423.000000,80.823729,1,0
3,4980,496.0,62,0,1,-0.166324,494.5,62.0,1.0,-0.204460,[1436],1,515.444444,415.777778,99.654435,1,0
4,4981,498.0,30,0,1,-2.091826,497.0,29.0,1.0,-2.203263,[1322],1,519.888889,425.000000,94.903121,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4972,4972,478.0,63,1,2,-0.092509,476.0,61.0,1.0,-0.031549,[376],2,417.333333,516.222222,98.758296,0,1
4973,4973,469.0,48,1,2,0.027891,466.5,46.0,1.0,-0.082354,[240],2,424.777778,519.666667,94.804051,0,1
4974,4974,454.0,37,0,2,-1.102093,461.5,32.0,1.0,-1.491509,[238],2,410.555556,517.333333,107.035335,0,1
4975,4975,475.0,64,1,2,2.034662,477.0,60.0,0.0,2.034873,[287],2,420.333333,514.111111,93.747349,0,1
